In [6]:
import sys
# Specify path to Remo
# Mac version
local_path_to_repo =  '/Users/melodi/Docs/remo-python'
# Windows version
#local_path_to_repo =  'C:/Users/crows/Documents/GitHub/remo-python'

sys.path.insert(0, local_path_to_repo)

In [7]:
%load_ext autoreload
%autoreload 2
import remo

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [8]:
remo.list_datasets() 

[Dataset 1 - 'Vova's dataset',
 Dataset 2 - 'open images detection',
 Dataset 3 - 'Subset of OID',
 Dataset 4 - 'train',
 Dataset 6 - 'test',
 Dataset 12 - 'oid 100',
 Dataset 13 - 'open images sample data',
 Dataset 19 - 'oid building ',
 Dataset 21 - 'oid building and car',
 Dataset 22 - 'oid building car detection',
 Dataset 25 - 'open images data',
 Dataset 26 - 'oid car',
 Dataset 28 - 'oid car and person',
 Dataset 29 - 'test upload',
 Dataset 30 - 'test upload 2',
 Dataset 31 - 'cats and dogs']

In [10]:
my_dataset = remo.get_dataset(28)
my_dataset.view()

Open http://localhost:8000/datasets/28


In [11]:
val = my_dataset[0:len(my_dataset) // 3]
train = my_dataset[len(my_dataset) // 3:]
val.export_annotation_to_csv('val.csv')
train.export_annotation_to_csv('train.csv')

We export our train and validation sets into .csv files in the format of:
  
    file_name,class,xmin,ymin,xmax,ymax,height,width
	039b687dbee6ec4a.jpg,Person,103.863296,283.794432,226.743296,588.800256,768,1024
	039b687dbee6ec4a.jpg,Person,383.268864,212.845824,558.811136,626.833920,768,1024
    ..


## Prepare for Object Detection

We will use darknet to simulate the usage flow. We need to convert annotation format into .xml for darknet.

In [36]:
from example.detection import *
import warnings
warnings.filterwarnings('ignore')

We will group by file_name and aggreagate in a list in the case of multiple objects.

In [34]:
prepare_data(path_to_csv_annotations='train.csv', image_path='/Users/melodi/Docs/images/', annotation_path='/Users/melodi/Docs/annots/')

### Darknet

Now we train our model with our dataset and feed our validation set into the model and prepare the results to upload Remo.

In [19]:
train_yolo(model_cfg="/Users/melodi/remo-python/cfg/yolov2.cfg", weights="/Users/melodi/remo-python/bin/yolov2.weights", batch_size=40, epoch=1, annotation_path="/Users/melodi/Docs/annots/", image_path="/Users/melodi/Docs/images/", label_path="/Users/melodi/remo-python/labels.txt", backup_path="/Users/melodi/remo-python/ckpt/")
results = predict_yolo(model_cfg="/Users/melodi/remo-python/cfg/yolov2.cfg", load=3, threshold=0.5, label_path="/Users/melodi/remo-python/labels.txt", backup_path="/Users/melodi/remo-python/ckpt/", root_path='/Users/melodi/remo-python/val/', classes=['Car','Person'])


/Users/melodi/remo-python/cfg/yolov2.cfg parsing /Users/melodi/Docs/annots/
Parsing for ['person', 'bicycle', 'car', 'motorbike', 'aeroplane', 'bus', 'train', 'truck', 'boat', 'traffic light', 'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow', 'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee', 'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard', 'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple', 'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'sofa', 'pottedplant', 'bed', 'diningtable', 'toilet', 'tvmonitor', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone', 'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear', 'hair drier', 'toothbrush'] 
[====================>]100%  09e3c15d734763b

### Visualize Predictions in Remo

Now we just post process the results in .json to have format .csv. We will upload this .csv file into Remo dataset.

In [39]:
df = post_process(results)
df.to_csv('/Users/melodi/Docs/remo-python/example/detection_preds_raw1.csv',index=False)

In [42]:
my_dataset.create_annotation_set(annotation_task='Object detection', name='Predictions',classes=['Person','Car'], path_to_annotation_file='/Users/melodi/Docs/remo-python/example/detection_preds_raw1.csv')

In [43]:
my_dataset.view()

Open http://localhost:8000/datasets/28
